In [13]:
import ixmp
from jaydebeapi import connect

platform = "ixmp_dev"
# model = "MESSAGEix-Materials"
# scenario = "2.0deg_petro_thesis_2_final_macro"
model = "MESSAGEix-GLOBIOM 1.1-R12-MAGPIE"
model = "MESSAGEix-GLOBIOM 1.1-R12-MAGPIE-MP50BD0BI78"
scenario = "600f_MP00BD0BI78_test_calib_macro"
#scenario = "2.0deg_no_SDGs_petro_thesis_1_macro"
scenario = "baseline"

# Connect to the platform
# This also starts the JVM with the correct classpath to find ojdbc*.jar
mp = ixmp.Platform(platform)
df = mp.scenario_list()
version = df[df["scenario"]==scenario]["version"]
version

162      8
167     10
180      1
206      2
208      1
        ..
9019     1
9030     1
9042     1
9055     1
9059     1
Name: version, Length: 192, dtype: int64

In [14]:
version = df[(df["scenario"]==scenario) & (df["model"]==model)]["version"]

In [15]:
version

7019    1
Name: version, dtype: int64

In [16]:
version = version.values[0]

In [17]:
#version = 7


conn = connect(
    "oracle.jdbc.driver.OracleDriver",
    "jdbc:oracle:thin:@x8oda.iiasa.ac.at:1521/PIXMP2.iiasa.ac.at",
    driver_args=["ixmp_dev", "ixmp_dev"]
)
curs = conn.cursor()

curs.execute(f"""
  SELECT RUN.id, status, lock_user, lock_date
  FROM
	RUN
	JOIN MODEL ON RUN.MODEL_ID = MODEL.ID
	JOIN SCENARIO ON RUN.SCEN_ID = SCENARIO.ID
  WHERE
    MODEL.NAME = {model!r}
	AND SCENARIO.NAME = {scenario!r}
	AND VERSION = {version}"""
             )
print(curs.fetchall())
conn.close()

[(37485, 0, 'maczek', '2023-08-28 08:53:34')]


In [19]:
mp._backend.jobj.unlockRunid(37485)
#for i in mp._backend.get_scenarios(False, "Materials", "baseline"):
#    mp._backend.check_out(i, False)